In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import pickle
import re
import math

In [3]:
"""
input: year 
get links from every matchday
Dict: key= matchday value= list of links
output: dict
"""
def Saisonliste(jahr):
    
    start= "https://www.kicker.de/bundesliga/spieltag/"
    urlstart= "https://www.kicker.de"
    if jahr==10:
        jahr2="09"
    else:
        jahr2= int(jahr)-1
    link_zusatz= "20"+str(jahr2)+"-"+str(jahr)+"/"
    url=start+link_zusatz
    dictionary={}
    count=1
    for i in range(1,35):
        link=url+str(i)
        spieltag= spieltage(link)
        mannschaftsliste={}
        for spiel in spieltag:
            newurl= urlstart+spiel
            mannschaft1= spiel.split("-")[0][1:]
            mannschaft2= spiel.split("-")[2]
            
            aufstellung= Aufstellung(newurl,mannschaft1,mannschaft2)
            #print(aufstellung)
            count+=1
            mannschaftsliste={**mannschaftsliste,**aufstellung}
            
        dictionary.update({i:mannschaftsliste})
    return dictionary 
"""
input: url
get links for each game
output: list of links
"""
def spieltage(url):
    liste=[]
    req = requests.get(url)
    soup = BeautifulSoup(req.text, "html.parser")
    spiele = soup.find_all("a",{"class":["kick__v100-gameList__gameRow__stateCell__indicator kick__v100-gameList__gameRow__stateCell__indicator--schema","kick__v100-gameList__gameRow__stateCell__indicator kick__v100-gameList__gameRow__stateCell__indicator--schema kick__v100-gameList__gameRow__stateCell__indicator--media"]})
    for i in spiele:
        link=i.get('href')
        link= link.replace("analyse","aufstellung")
        liste.append(link)
    return liste
"""
input: url, team1, team2
get lineup (names of players)
output: dic
"""
def Aufstellung(url,mannschaft1,mannschaft2):
    dic= {}
    spielerliste=[]
    req = requests.get(url)
    
    if req.ok:
        #print(mannschaft1)
        soup = BeautifulSoup(req.text, "html.parser")
        spieler= soup.findAll("span","kick__lineup-player-card__name")
        for name in spieler:
            spielerliste.append(str(name.getText()).strip())
        #print(spielerliste[:11])
        #print(spielerliste[11:])
        dic.update({mannschaft1:spielerliste[:11]})
        dic.update({mannschaft2:spielerliste[11:]})

    
    return dic
#Aufstellung("https://www.kicker.de/bayern-gegen-schalke-2020-bundesliga-4667048/aufstellung","bayern","schalke")
    


In [1]:
"""
input: dict, year
create a pkl with data of season
ouput: pkl file
"""

def pkl_erstellen(result,jahr):
    data= pd.DataFrame.from_dict(result)
    #print(data[2])
    filename= "AufstellungenNeu"+str(jahr)+".pkl"
    file = open(filename, 'wb')
    pickle.dump(data,file)
    file.close()
    
#for i in range(10,21):
#    result= Saisonliste(i)
#    pkl_erstellen(result,i)